In [ ]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import matplotlib.pyplot as plt
import shutil
import cv2
import math
import fnmatch
import imutils
from google.colab.patches import cv2_imshow
from decimal import Decimal
from IPython.display import HTML
from base64 import b64encode
import PIL
from PIL import Image
print(cv2.__version__)

In [ ]:
!pip uninstall h5py
!pip install h5py==3.9.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive/') #by default gdrive path will be /content/drive

In [ ]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

In [ ]:
!sudo apt-get install -qq w3m # to act as web browser
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive2
%cd drive2
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive2/MyDrive

TestCases : Real Single Face Video , Real Multiple faces , Fake Single face Video, Multiple face fake video 
use these bellow code cells for the testcases as a template by changing the actual class and input path  

In [ ]:
inputvidpath='Give the path of the input video 


In [ ]:
actualclass='real'

In [ ]:
vidname= inputvidpath.split("/")[5]
print(vidname)

In [ ]:
mp4 = open(inputvidpath,'rb').read()   #rb-reading a binary file like an image file and not a text file
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
path="/content/drive/MyDrive/sample-images/"+vidname

In [ ]:
#Frame split-up module

import cv2


def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite("image"+str(count)+".jpg", image) # save frame as JPG file in the current directory
        #plt.imshow(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #plt.imshow(image)
    return hasFrames


vidcap = cv2.VideoCapture(inputvidpath)
sec = 0
frameRate = 0.10 #10 frames per second
count=1
vidname= inputvidpath.split("/")[5]
path="/content/drive/MyDrive/sample-images/"+vidname
if os.path.exists(path):
   shutil.rmtree(path, ignore_errors=True); #if path already exists delete it
os.makedirs(path);
os.chdir(path)#change the current directory to the path
success = getFrame(sec)
while success:
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)

In [ ]:
l=len(os.listdir(path))
print(l)
print(path)

In [ ]:
#Finding keyframes for extracted frames
diff=[]
imageA = cv2.imread( path+'/image'+str(1)+'.jpg')
grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
diff.append(cv2.absdiff(grayA, grayA))
for i in range(1,l):#loops till l-1. For 7 images 6 comparisons are needed
  imageA = cv2.imread( path+'/image'+str(i)+'.jpg')
  imageB = cv2.imread(path+'/image'+str(i+1)+'.jpg')
  #print(i,":::",imageA.shape)
  #print(i+1,":::",imageB.shape)
  grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
  grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
  diff.append(cv2.absdiff(grayA, grayB))
 #print(diff)
mn = np.mean(diff)
st_d = np.std(diff)
 #print(mn,st_d)
a = 4
ts = mn + (a * st_d)
 #print('The threshold==>',ts)
print("length of diff array:",len(diff))
a_fr = []#Creating an empty list
for i in range(len(diff)):
    mn = np.mean(diff[i])#Calculating the mean for each frame
    st_d = np.std(diff[i])
    fr_ts = mn + (4*st_d)#Finding the threshold values for each frame/image
    #print(i,fr_ts)
    a_fr.append([i,fr_ts])
keyframes = []
for i,ac_tr in(a_fr):
    if ac_tr >= ts:
        #print(i,ac_tr)
        keyframes.append(i)
print(keyframes)
print("Length:",len(keyframes))
 #Renaming
x=len(fnmatch.filter(os.listdir(path), 'key*.jpg'))
if(x==0):   #if no keyframe has been previously created, create keyframes
  for i in range(0,len(keyframes)):
   oldname=path+"/image"+str(keyframes[i])+".jpg"
   newname=path+"/key"+str(i+1)+".jpg"
   os.rename(oldname, newname)
#Check
print("check:", len(fnmatch.filter(os.listdir(path), 'key*.jpg'))    )

In [ ]:
#To remove un-necessary frames after finding keyframes
import glob
#Add proper file path
removefilelist=glob.glob(path+"/**/image*.jpg", recursive=True) # all imgs(not keyframes) from all videos
for filePath in removefilelist:
    try:
        os.remove(filePath)
        #print(filePath)
    except OSError:
        print("Error while deleting file",filePath)

In [ ]:
cols=5
rows=math.ceil(len(keyframes)/cols)
print(rows)

In [ ]:
fig=plt.figure(figsize=(16,10))
cols=5
rows=math.ceil(len(keyframes)/cols)
for j in range(0,rows*cols):
 fig.add_subplot(rows,cols,j+1)

In [ ]:
fig=plt.figure(figsize=(16,10))
cols=5
rows=math.ceil(len(keyframes)/cols)
for j in range(0,len(keyframes)):
 fig.add_subplot(rows,cols,j+1)
 img=cv2.imread(path+"/key"+str(j+1)+".jpg")
 img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
 plt.imshow(img)
 plt.title("keyFrame"+str(j+1))
 plt.axis('off')

In [ ]:
import seaborn as sns
%matplotlib inline

from matplotlib.patches import Rectangle
from tensorflow.python.keras.layers import Conv2D, Input, ZeroPadding2D, Dense, Lambda
from tensorflow.python.keras.models import Model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import math
import numpy as np
import cv2
from tensorflow.keras import Sequential

In [ ]:
import tensorflow as tf

def load_mobilenetv2_224_075_detector(path):


  input_tensor = tf.keras.Input(shape=(224, 224, 3))
  output_tensor = MobileNetV2(weights=None, include_top=False, input_tensor=input_tensor, alpha=0.75).output
  output_tensor = tf.keras.layers.ZeroPadding2D()(output_tensor)
  output_tensor = tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=5)(output_tensor)

  model = tf.keras.Model(inputs=input_tensor, outputs=output_tensor)

  try:
    model.load_weights(path)
  except ValueError as e:
    # If the model architecture does not match the weight file architecture,
    # update the model architecture to match the weight file architecture.

    if 'You are trying to load a weight file containing 106 layers into a model with 1 layers.' in e:
      model.compile(optimizer='adam')

  return model

mobilenetv2 = load_mobilenetv2_224_075_detector("/content/drive/MyDrive/deepfake/facedetection-mobilenetv2-size224-alpha0.75.h5")

In [ ]:
# Converts A:B aspect rate to B:A
def transpose_shots(shots):
    return [(shot[1], shot[0], shot[3], shot[2], shot[4]) for shot in shots]

#That constant describe pieces for 16:9 images
SHOTS = {
    # fast less accurate
    '2-16/9' : {
        'aspect_ratio' : 16/9,
        'shots' : [
             (0, 0, 9/16, 1, 1),
             (7/16, 0, 9/16, 1, 1)
        ]
    },
    # slower more accurate
    '10-16/9' : {
        'aspect_ratio' : 16/9,
        'shots' : [
             (0, 0, 9/16, 1, 1),
             (7/16, 0, 9/16, 1, 1),
             (0, 0, 5/16, 5/9, 0.5),
             (0, 4/9, 5/16, 5/9, 0.5),
             (11/48, 0, 5/16, 5/9, 0.5),
             (11/48, 4/9, 5/16, 5/9, 0.5),
             (22/48, 0, 5/16, 5/9, 0.5),
             (22/48, 4/9, 5/16, 5/9, 0.5),
             (11/16, 0, 5/16, 5/9, 0.5),
             (11/16, 4/9, 5/16, 5/9, 0.5),
        ]
    }
}

# 9:16 respectively
SHOTS_T = {
    '2-9/16' : {
        'aspect_ratio' : 9/16,
        'shots' : transpose_shots(SHOTS['2-16/9']['shots'])
    },
    '10-9/16' : {
        'aspect_ratio' : 9/16,
        'shots' : transpose_shots(SHOTS['10-16/9']['shots'])
    }
}

def r(x):
    return int(round(x))

def sigmoid(x):
    return 1 / (np.exp(-x) + 1)

def non_max_suppression(boxes, p, iou_threshold):

    if len(boxes) == 0:
        return np.array([])

    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    indexes = np.argsort(p)
    true_boxes_indexes = []

    while len(indexes) > 0:
        true_boxes_indexes.append(indexes[-1])

        intersection = np.maximum(np.minimum(x2[indexes[:-1]], x2[indexes[-1]]) - np.maximum(x1[indexes[:-1]], x1[indexes[-1]]), 0) * np.maximum(np.minimum(y2[indexes[:-1]], y2[indexes[-1]]) - np.maximum(y1[indexes[:-1]], y1[indexes[-1]]), 0)
        iou = intersection / ((x2[indexes[:-1]] - x1[indexes[:-1]]) * (y2[indexes[:-1]] - y1[indexes[:-1]]) + (x2[indexes[-1]] - x1[indexes[-1]]) * (y2[indexes[-1]] - y1[indexes[-1]]) - intersection)

        indexes = np.delete(indexes, -1)
        indexes = np.delete(indexes, np.where(iou >= iou_threshold)[0])

    return boxes[true_boxes_indexes]

def union_suppression(boxes, threshold):
    if len(boxes) == 0:
        return np.array([])

    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    indexes = np.argsort((x2 - x1) * (y2 - y1))
    result_boxes = []

    while len(indexes) > 0:
        intersection = np.maximum(np.minimum(x2[indexes[:-1]], x2[indexes[-1]]) - np.maximum(x1[indexes[:-1]], x1[indexes[-1]]), 0) * np.maximum(np.minimum(y2[indexes[:-1]], y2[indexes[-1]]) - np.maximum(y1[indexes[:-1]], y1[indexes[-1]]), 0)
        min_s = np.minimum((x2[indexes[:-1]] - x1[indexes[:-1]]) * (y2[indexes[:-1]] - y1[indexes[:-1]]), (x2[indexes[-1]] - x1[indexes[-1]]) * (y2[indexes[-1]] - y1[indexes[-1]]))
        ioms = intersection / (min_s + 1e-9)
        neighbours = np.where(ioms >= threshold)[0]
        if len(neighbours) > 0:
            result_boxes.append([min(np.min(x1[indexes[neighbours]]), x1[indexes[-1]]), min(np.min(y1[indexes[neighbours]]), y1[indexes[-1]]), max(np.max(x2[indexes[neighbours]]), x2[indexes[-1]]), max(np.max(y2[indexes[neighbours]]), y2[indexes[-1]])])
        else:
            result_boxes.append([x1[indexes[-1]], y1[indexes[-1]], x2[indexes[-1]], y2[indexes[-1]]])

        indexes = np.delete(indexes, -1)
        indexes = np.delete(indexes, neighbours)

    return result_boxes

class FaceDetector():

    def __init__(self, model=mobilenetv2, shots=[SHOTS['10-16/9'], SHOTS_T['10-9/16']], image_size=224, grids=7, iou_threshold=0.1, union_threshold=0.1, prob_threshold=0.65):
        self.model = model
        self.shots = shots
        self.image_size = image_size
        self.grids = grids
        self.iou_threshold = iou_threshold
        self.union_threshold = union_threshold
        self.prob_threshold = prob_threshold


    def detect(self, frame):
        original_frame_shape = frame.shape

        aspect_ratio = None
        for shot in self.shots:
            if abs(frame.shape[1] / frame.shape[0] - shot["aspect_ratio"]) < 1e-9:
                aspect_ratio = shot["aspect_ratio"]
                shots = shot

        assert aspect_ratio is not None

        c = min(frame.shape[0], frame.shape[1] / aspect_ratio)
        slice_h_shift = r((frame.shape[0] - c) / 2)
        slice_w_shift = r((frame.shape[1] - c * aspect_ratio) / 2)
        if slice_w_shift != 0 and slice_h_shift == 0:
            frame = frame[:, slice_w_shift:-slice_w_shift]
        elif slice_w_shift == 0 and slice_h_shift != 0:
            frame = frame[slice_h_shift:-slice_h_shift, :]

        frames = []
        for s in shots["shots"]:
            frames.append(cv2.resize(frame[r(s[1] * frame.shape[0]):r((s[1] + s[3]) * frame.shape[0]), r(s[0] * frame.shape[1]):r((s[0] + s[2]) * frame.shape[1])], (self.image_size, self.image_size), interpolation=cv2.INTER_NEAREST))
        frames = np.array(frames)

        predictions = self.model.predict(frames, batch_size=len(frames), verbose=0)
        predictions.shape


        boxes = []
        prob = []
        shots = shots['shots']
        for i in range(len(shots)):
            slice_boxes = []
            slice_prob = []
            for j in range(predictions.shape[1]):
              if len(predictions.shape) > 2:
                for k in range(predictions.shape[2]):



                    p = sigmoid(predictions[i][j][k][4])
                    if not(p is None) and p > self.prob_threshold:
                        px = sigmoid(predictions[i][j][k][0])
                        py = sigmoid(predictions[i][j][k][1])
                        pw = min(math.exp(predictions[i][j][k][2] / self.grids), self.grids)
                        ph = min(math.exp(predictions[i][j][k][3] / self.grids), self.grids)
                        if not(px is None) and not(py is None) and not(pw is None) and not(ph is None) and pw > 1e-9 and ph > 1e-9:
                            cx = (px + j) / self.grids
                            cy = (py + k) / self.grids
                            wx = pw / self.grids
                            wy = ph / self.grids
                            if wx <= shots[i][4] and wy <= shots[i][4]:
                                lx = min(max(cx - wx / 2, 0), 1)
                                ly = min(max(cy - wy / 2, 0), 1)
                                rx = min(max(cx + wx / 2, 0), 1)
                                ry = min(max(cy + wy / 2, 0), 1)

                                lx *= shots[i][2]
                                ly *= shots[i][3]
                                rx *= shots[i][2]
                                ry *= shots[i][3]

                                lx += shots[i][0]
                                ly += shots[i][1]
                                rx += shots[i][0]
                                ry += shots[i][1]

                                slice_boxes.append([lx, ly, rx, ry])
                                slice_prob.append(p)

            slice_boxes = np.array(slice_boxes)
            slice_prob = np.array(slice_prob)

            slice_boxes = non_max_suppression(slice_boxes, slice_prob, self.iou_threshold)

            for sb in slice_boxes:
                boxes.append(sb)


        boxes = np.array(boxes)
        boxes = union_suppression(boxes, self.union_threshold)

        for i in range(len(boxes)):
            boxes[i][0] /= original_frame_shape[1] / frame.shape[1]
            boxes[i][1] /= original_frame_shape[0] / frame.shape[0]
            boxes[i][2] /= original_frame_shape[1] / frame.shape[1]
            boxes[i][3] /= original_frame_shape[0] / frame.shape[0]

            boxes[i][0] += slice_w_shift / original_frame_shape[1]
            boxes[i][1] += slice_h_shift / original_frame_shape[0]
            boxes[i][2] += slice_w_shift / original_frame_shape[1]
            boxes[i][3] += slice_h_shift / original_frame_shape[0]

        return list(boxes)

In [ ]:
def setaspectratio( filepath ):
  from PIL import Image
  from numpy import size
  from numpy import asarray
  image  = Image.open(filepath )
  #image= cv2.imread(ff_image_path+"/"+imgvid[3]+"/key2.jpg")
  width  = image.size[0]
  height = image.size[1]
  #print(width,height)
  aspect = width / float(height)

  ideal_width = 1280
  ideal_height = 720
  if(width != ideal_width or height!= ideal_height):
   ideal_aspect = ideal_width / float(ideal_height)

   if aspect > ideal_aspect:
    # Then crop the left and right edges:
    new_width = int(ideal_aspect * height)
    offset = (width - new_width) / 2
    resize = (offset, 0, width - offset, height)
   else:
    # ... crop the top and bottom:
    new_height = int(width / ideal_aspect)
    offset = (height - new_height) / 2
    resize = (0, offset, width, height - offset)

   thu = image.crop(resize)
   thu=thu.resize((ideal_width, ideal_height), Image.Resampling.LANCZOS)
   #model.fit(X_train, y_train, epochs=10)


   ffff= asarray(thu)
   return ffff
  else:
   ffff= asarray(image)
   return ffff


In [ ]:
detector = FaceDetector()

In [ ]:
detector = FaceDetector()

In [ ]:
# A sample face detection image
#frame=cv2.imread(path+"/key1.jpg")
#frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
frame=setaspectratio(path+"/key1.jpg")
rgbframe = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.imshow(frame)
dim=(299,299)
boxes = detector.detect(frame)
 # lets's draw boxes, just multiply each predicted [0, 1] relative coordinate to image side in pixels respectively
for box in boxes:
        lx = int(round(box[0] * frame.shape[1]))
        ly = int(round(box[1] * frame.shape[0]))
        rx = int(round(box[2] * frame.shape[1]))
        ry = int(round(box[3] * frame.shape[0]))
        # x, y, w, h here
        ax.add_patch(Rectangle((lx,ly),rx - lx,ry - ly,linewidth=2,edgecolor='r',facecolor='none'))

In [ ]:
#Face extraction and storage

#dim=(w,h)
dim=(299,299)
b=1
storagepath="/content/drive/MyDrive/sample-faces/"+vidname   #path to store extracted faces
if os.path.exists(storagepath):
   shutil.rmtree(storagepath, ignore_errors=True); #if path already exists delete it
os.makedirs(storagepath);
os.chdir(storagepath)
for keyframe in os.listdir(path):
    frame=setaspectratio(path+"/"+keyframe)
    rgbframe = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #frame=cv2.imread(path+"/"+keyframe)
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    boxes = detector.detect(rgbframe)
    # lets's draw boxes, just multiply each predicted [0, 1] relative coordinate to image side in pixels respectively
    for box in boxes:
        lx = int(round(box[0] * frame.shape[1]))
        ly = int(round(box[1] * frame.shape[0]))
        rx = int(round(box[2] * frame.shape[1]))
        ry = int(round(box[3] * frame.shape[0]))
        # x, y, w, h here
        #ax.add_patch(Rectangle((lx,ly),rx - lx,ry - ly,linewidth=2,edgecolor='r',facecolor='none'))
        frame2 = frame[ly:ry, lx:rx]
        rgb = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
        face = cv2.resize(rgb, dim, interpolation=cv2.INTER_LANCZOS4)
        cv2.imwrite("face"+str(b)+".jpg", face)#Detected face
        b=b+1

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
#Handling Image data Generators
test_data_gen = ImageDataGenerator()

In [ ]:
from keras.models import load_model
newmodel=load_model("/content/drive/MyDrive/deepfake/ChkpointXception10weights.h5")

In [ ]:
extractedfaces=os.listdir("/content/drive/MyDrive/sample-faces/"+vidname)
print(extractedfaces)

In [ ]:
whatdata=[]
facepath="/content/drive/MyDrive/sample-faces/"+vidname
if os.path.exists(facepath):
    for filename in extractedfaces:
      if filename.endswith("jpg"):
        # Your code comes here such as
        whatdata.append(facepath+"/"+filename)
print(whatdata)

In [ ]:
#creating DataFrame
videodf = pd.DataFrame(whatdata, columns=['path'])
bs= len(videodf)
print(bs)

In [ ]:
videodf['path'] = whatdata


In [ ]:
video_imgs = test_data_gen.flow_from_dataframe(dataframe=videodf,x_col='path', target_size=(299,299), class_mode=None,batch_size=bs, shuffle=False)

video_prediction = newmodel.predict(video_imgs)

predictclass=[]
for i in range(0, len(video_prediction)):
  predictclass.append(  str(video_prediction[i][:2])  )
# for value in predictclass:
#  print(value)

In [ ]:
from keras.models import load_model
newmodel=load_model("/content/drive/MyDrive/deepfake/ChkpointXception10weights.h5")

extractedfaces=os.listdir("/content/drive/MyDrive/sample-faces/"+vidname)
print(extractedfaces)

whatdata=[]
facepath="/content/drive/MyDrive/sample-faces/"+vidname
if os.path.exists(facepath):
    for filename in extractedfaces:
      if filename.endswith("jpg"):
        # Your code comes here such as
        whatdata.append(facepath+"/"+filename)
print(whatdata)

#creating DataFrame
videodf = pd.DataFrame(whatdata, columns=['path'])
videodf['path'] = videodf['path'].apply(lambda x: str(x))
bs= len(videodf)
print(bs)

video_imgs = test_data_gen.flow_from_dataframe(dataframe=videodf,x_col='path', target_size=(299,299), class_mode=None,batch_size=bs, shuffle=False)

video_prediction = newmodel.predict(video_imgs)

predictclass=[]
for i in range(0, len(video_prediction)):
  predictclass.append(  str(video_prediction[i][:2])  )
# for value in predictclass:
#  print(value)


In [ ]:

video_output = np.argmax(video_prediction, axis=1)
print(video_output[0:25])
avg = video_output.mean()
print(avg)
if avg >= 0.5 :
   predictedclass='real'
elif avg <0.5:
   predictedclass='fake'

In [ ]:
print("ACTUAL:"+actualclass)
print("PREDICTED :" +predictedclass)